# Music Recommendation Project

Short description here... this will be Data Pre-processing: Import, exploration, cleaning, feature engineering, resulting in saving off feature data ... [data available on Kaggle](https://www.kaggle.com/c/kkbox-music-recommendation-challenge/data)

Author: Ben Walsh \
February 7, 2020

## Contents

1. [Data Import](#data-import)
2. Data Exploration
3. Data Cleaning
4. Feature Engineering
5. Saving Data

## <a class="anchor" id="data-import"></a>1. Data Import

### Import libraries

In [1]:
import pandas as pd
import os

First import all data: input traiing data, memeber data, and song data

In [10]:
member_datafile = './data-input/members.csv'

### Import member data

In [13]:
if os.path.exists(member_datafile):
    member_data = pd.read_csv(member_datafile)
else:
    print('Member data file {} not found!'.format(member_datafile))

## 2. Data Exploration

### Member Data

In [14]:
member_data.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [16]:
member_data.describe()

,city,bd,registered_via,registration_init_time,expiration_date
count,34403.000000,34403.000000,34403.000000,3.440300e+04,3.440300e+04
mean,5.371276,12.280935,5.953376,2.013994e+07,2.016901e+07
std,6.243929,18.170251,2.287534,2.954015e+04,7.320925e+03
min,1.000000,-43.000000,3.000000,2.004033e+07,1.970010e+07
25%,1.000000,0.000000,4.000000,2.012103e+07,2.017020e+07
50%,1.000000,0.000000,7.000000,2.015090e+07,2.017091e+07
75%,10.000000,25.000000,9.000000,2.016110e+07,2.017093e+07
max,22.000000,1051.000000,16.000000,2.017023e+07,2.020102e+07


### Observations

 ----- Any?  ------